# Persiapan Library

In [21]:
%pip install -q pyspellchecker
%pip install -q Sastrawi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd
import pickle

from collections import Counter

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk

# 1. Stop Word
nltk.download('stopwords')
from nltk.corpus import stopwords

# 3. Cek Ejaan Pembakuan kata B.inggris
from spellchecker import SpellChecker
# Inisialisasi SpellChecker untuk bahasa Indonesia
spell = SpellChecker(language='en')

# 4.Stemming
## Indo
nltk.download('wordnet')
# Menggunaakn Sastrawi untuk memproses teks dalam bahasa Indonesia.
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()

## English
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()

# 5. Tokenisasi
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Praproses Teks

## Membuat Fungsi

### Menghilangkan symbol atau tanda baca

In [23]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\[\]\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords Indonesia
def clean_stopwords_id(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Menghapus stopwords Inggris
def clean_stopwords_en(text):
    stopword = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

# Stemming with Porter Stemmer
def porter_stemming(text):
    # Memecah teks menjadi kata-kata
    words = nltk.word_tokenize(text)  # Tokenisasi
    # Melakukan stemming pada setiap kata
    stemmed_words = [stemmer.stem(word) for word in tqdm(words)]
    # Menggabungkan kata-kata yang telah di-stem
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

# Fungsi untuk memperbaiki ejaan dalam bahasa Inggris dengan tqdm
def correct_spelling_english(text):
    if not isinstance(text, str):
        return text  # Mengabaikan nilai non-string
    words = text.split()
    corrected_words = []
    # Menggunakan tqdm untuk menampilkan progres perbaikan ejaan
    for word in tqdm(words, desc="Fix Spell", unit="kata"):
        corrected = spell.correction(word)  # Periksa ejaan kata
        # Jika spell.correction mengembalikan None, gunakan kata aslinya atau string kosong
        if corrected is None:
            corrected_words.append(word)
        else:
            corrected_words.append(corrected)

    corrected_text = ' '.join(corrected_words)
    return corrected_text

# Fungsi untuk melakukan tokenisasi pada teks
def tokenize_text(text):
    if isinstance(text, str):  # Memastikan input adalah string
        return word_tokenize(text)  # Mengembalikan token
    return []  # Kembali ke list kosong jika bukan string


## PTA Berita Online Kompas.com

### Load Data

#### Data PTA

In [24]:
main_df_berita = pd.read_csv('datasets/kompas_news_data.csv', delimiter=',')
main_df_berita.head()

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,91 Orang Diduga Masih Tertimbun Reruntuhan Pon...,"SIDOARJO, KOMPAS.com - Sebanyak 91 orang diper...",01/10/2025,REGIONAL
1,2,"OpenAI Rilis Sora 2, Model AI Pembuat Video Re...","KOMPAS.com - Induk ChatGPT, OpenAI, merilis mo...",01/10/2025,TEKNO
2,3,"Pimpin Upacara Hari Kesaktian Pancasila, Pramo...","JAKARTA, KOMPAS.com - Gubernur DKI Jakarta Pra...",01/10/2025,NEWS
3,4,"Dedie Rachim Hadiri Rakornas PSEL, Dukung Peng...","KOMPAS.com – Wali Kota Bogor, Dedie A Rachim m...",01/10/2025,ADVERTORIAL
4,5,12 Resep Masakan Modal Rp 10.000 untuk Sehari-...,KOMPAS.com - Makan enak tidak selalu harus mah...,01/10/2025,FOOD


In [25]:
main_df_berita['Kategori Berita'].value_counts()

Kategori Berita
NEWS           2129
REGIONAL       1878
MONEY           898
PROV            729
BOLA            626
OTOMOTIF        554
TREN            488
HYPE            483
GLOBAL          304
PROPERTI        269
TEKNO           232
EDUKASI         229
CAHAYA          168
LIFESTYLE       165
LESTARI         126
FOOD            119
TRAVEL          105
ADVERTORIAL      93
CEK FAKTA        91
SAINS            76
SKOLA            69
HOMEY            62
HEALTH           56
IKN              18
KATANETIZEN      14
STORI             4
AGRI              4
UMKM              1
Name: count, dtype: int64

In [26]:
KATEGORI_TARGET = ['NEWS', 'REGIONAL', 'MONEY', 'PROV', 'BOLA', 'OTOMOTIF']
SAMPLE_SIZE = 200

# 1. Filter data untuk HANYA 6 kategori target
df_filtered = main_df_berita[main_df_berita['Kategori Berita'].isin(KATEGORI_TARGET)]

# 2. Fungsi untuk mengambil sample sebanyak 200 (atau kurang jika tidak mencapai 200)
def sample_n(df):
    return df.sample(n=min(SAMPLE_SIZE, len(df)), random_state=42)

# 3. Ambil sample 200 dari setiap kategori yang sudah difilter
df_final_sample = (
    df_filtered
    .groupby('Kategori Berita')
    .apply(sample_n)
    .reset_index(drop=True)
)

# Cek hasil akhir
# print("--- Jumlah Berita per Kategori pada Hasil Sample Akhir ---")
# print(df_final_sample['Kategori Berita'].value_counts())
# print(f"\nTotal Data Akhir: {len(df_final_sample)}")
df_final_sample

C:\Users\willy\AppData\Local\Temp\ipykernel_22472\3761331255.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(sample_n)


,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,6973,Pengamat Menilai Gebrakan Awal Erick Thohir Se...,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,24/09/2025,BOLA
1,7835,"Enzo Maresca Sebut Liverpool Sulit Dikejar, Si...","KOMPAS.com - Manajer Chelsea, Enzo Maresca men...",23/09/2025,BOLA
2,3234,Kata-kata Arne Slot Usai Liverpool Telan Kekal...,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...",28/09/2025,BOLA
3,3216,Hasil dan Klasemen Liga Italia: Beda Nasib Juv...,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,28/09/2025,BOLA
4,7654,"Dana Tersedia Rp 10 Miliar, Erick Thohir Bicar...",KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,23/09/2025,BOLA
...,...,...,...,...,...
1195,3992,"Diprotes Warga karena Bising, Dapur SPPG di Ka...","KARANGANYAR, KOMPAS.com - Dapur Satuan Pelayan...",26/09/2025,REGIONAL
1196,8395,Bukan Gunung Meletus! BMKG Ungkap Penyebab Awa...,"BANDUNG, KOMPAS.com -Kepala Stasiun Geofisika ...",23/09/2025,REGIONAL
1197,2606,"Cegah Keracunan Massal, Pemkot Tegal Libatkan ...","TEGAL, KOMPAS.com – Maraknya kasus keracunan m...",29/09/2025,REGIONAL
1198,4244,Pemkot Solo Blokir 5 Akun Medsos Bermuatan Pen...,"SOLO, KOMPAS.com - Pemerintah Kota (Pemkot) So...",26/09/2025,REGIONAL


#### Remove data Missing

In [27]:
main_df_berita = df_final_sample[["Isi Berita"]].dropna()
main_df_berita.isnull().sum()
main_df_berita.head()

,Isi Berita
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men..."
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men..."
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...


#### Lower case

In [28]:
main_df_berita['lwr_indo'] = main_df_berita['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df_berita[["Isi Berita","lwr_indo"]])
casefolding

,Isi Berita,lwr_indo
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,kompas.com - menteri pemuda dan olahraga erick...
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men...","kompas.com - manajer chelsea, enzo maresca men..."
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...","kompas.com - pelatih liverpool, arne slot, men..."
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,kompas.com - hasil terbaru pekan kelima liga i...
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,kompas.com - menteri pemuda dan olahraga repub...
...,...,...
1195,"KARANGANYAR, KOMPAS.com - Dapur Satuan Pelayan...","karanganyar, kompas.com - dapur satuan pelayan..."
1196,"BANDUNG, KOMPAS.com -Kepala Stasiun Geofisika ...","bandung, kompas.com -kepala stasiun geofisika ..."
1197,"TEGAL, KOMPAS.com – Maraknya kasus keracunan m...","tegal, kompas.com – maraknya kasus keracunan m..."
1198,"SOLO, KOMPAS.com - Pemerintah Kota (Pemkot) So...","solo, kompas.com - pemerintah kota (pemkot) so..."


#### Clean Stopwords

In [29]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df_berita['clean_sw_indo'] = main_df_berita['lwr_indo'].apply(clean_stopwords_id)
stop_words = pd.DataFrame(main_df_berita[['lwr_indo','clean_sw_indo']])
stop_words

,lwr_indo,clean_sw_indo
0,kompas.com - menteri pemuda dan olahraga erick...,kompas.com - menteri pemuda olahraga erick tho...
1,"kompas.com - manajer chelsea, enzo maresca men...","kompas.com - manajer chelsea, enzo maresca men..."
2,"kompas.com - pelatih liverpool, arne slot, men...","kompas.com - pelatih liverpool, arne slot, men..."
3,kompas.com - hasil terbaru pekan kelima liga i...,kompas.com - hasil terbaru pekan liga italia 2...
4,kompas.com - menteri pemuda dan olahraga repub...,kompas.com - menteri pemuda olahraga republik ...
...,...,...
1195,"karanganyar, kompas.com - dapur satuan pelayan...","karanganyar, kompas.com - dapur satuan pelayan..."
1196,"bandung, kompas.com -kepala stasiun geofisika ...","bandung, kompas.com -kepala stasiun geofisika ..."
1197,"tegal, kompas.com – maraknya kasus keracunan m...","tegal, kompas.com – maraknya keracunan makanan..."
1198,"solo, kompas.com - pemerintah kota (pemkot) so...","solo, kompas.com - pemerintah kota (pemkot) so..."


#### Menghilangkan symbol atau tanda baca

In [30]:
main_df_berita['clean_stb_indo'] = main_df_berita['clean_sw_indo'].apply(clean_punct)
clean_punct_df = pd.DataFrame(main_df_berita[['clean_sw_indo','clean_stb_indo']])
clean_punct_df

,clean_sw_indo,clean_stb_indo
0,kompas.com - menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick tho...
1,"kompas.com - manajer chelsea, enzo maresca men...",kompas com manajer chelsea enzo maresca meng...
2,"kompas.com - pelatih liverpool, arne slot, men...",kompas com pelatih liverpool arne slot menga...
3,kompas.com - hasil terbaru pekan liga italia 2...,kompas com hasil terbaru pekan liga italia ...
4,kompas.com - menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik ...
...,...,...
1195,"karanganyar, kompas.com - dapur satuan pelayan...",karanganyar kompas com dapur satuan pelayana...
1196,"bandung, kompas.com -kepala stasiun geofisika ...",bandung kompas com kepala stasiun geofisika b...
1197,"tegal, kompas.com – maraknya keracunan makanan...",tegal kompas com maraknya keracunan makanan ...
1198,"solo, kompas.com - pemerintah kota (pemkot) so...",solo kompas com pemerintah kota pemkot solo ...


##### Cek Ejaan Peter Norvig

###### Fungsi

In [31]:
# Spell checker (default bahasa Inggris, kita isi dengan kosakata dataset CNN)
spell = SpellChecker(language=None)
unique_words = set(" ".join(main_df_berita["clean_stb_indo"].astype(str)).split())
print(unique_words)
spell.word_frequency.load_words(unique_words)

def spellcheck_text(text):
    if pd.isna(text):
        return ""
    tokens = text.split()
    corrected_tokens = []
    for token in tokens:
        if token not in spell:
            correction = spell.correction(token)
            corrected_tokens.append(correction if correction else token)
        else:
            corrected_tokens.append(token)
    return " ".join(corrected_tokens)

{'gozali', 'nantinya', 'ternama', 'setinggi', 'simanjuntak', 'menunggu', 'mengupayakan', 'pecal', 'ffws', 'bournemouth', 'guncang', 'randy', 'gemuruh', 'syukurnya', 'ditargetkan', 'penghujungnya', 'pertamaxmenurutnya', 'tidur', 'baiklah', 'menghanguskan', 'rafni', 'wamenkeu', 'diblokir', 'dicat', 'comharyanti', 'alami', 'landscape', 'melakukannya', 'dsp', 'gratisan', 'psikis', 'kerinduannya', 'mobil', 'humas', 'lansiran', 'bachtiar', 'menyelesaikannya', 'ratas', 'peltu', 'maskulin', 'melaksanakannya', 'kombes', 'mengunduh', 'rambut', 'coupe', 'bapakibu', 'jammalik', 'masif', 'akur', 'balangan', 'tambahkan', 'deja', 'keliling', 'roadside', 'lebarnya', 'saudaranya', 'uncovered', 'peleknya', 'scoopydampaknya', 'awreceh', 'bertuliskan', 'dipertahankan', 'gatot', 'circular', 'marathon', 'responsibility', 'kerajinan', 'jejeran', 'rejoagung', 'memantau', 'casc', 'note', 'mosquera', 'budiono', 'mild', 'pelalawan', 'merapikan', 'sedih', 'irakselain', 'teken', 'aren', 'josua', 'sawah', 'sirenese

#### Cek Ejaan Pembakuan Kata

In [32]:
# main_df_berita = main_df_berita.head(10)

In [33]:
main_df_berita['clean_typo_indo'] = main_df_berita['clean_stb_indo'].apply(spellcheck_text)
clean_spell=pd.DataFrame(main_df_berita[["clean_stb_indo",'clean_typo_indo']])
clean_spell

,clean_stb_indo,clean_typo_indo
0,kompas com menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick thohi...
1,kompas com manajer chelsea enzo maresca meng...,kompas com manajer chelsea enzo maresca mengan...
2,kompas com pelatih liverpool arne slot menga...,kompas com pelatih liverpool arne slot mengaku...
3,kompas com hasil terbaru pekan liga italia ...,kompas com hasil terbaru pekan liga italia per...
4,kompas com menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik in...
...,...,...
1195,karanganyar kompas com dapur satuan pelayana...,karanganyar kompas com dapur satuan pelayanan ...
1196,bandung kompas com kepala stasiun geofisika b...,bandung kompas com kepala stasiun geofisika bm...
1197,tegal kompas com maraknya keracunan makanan ...,tegal kompas com maraknya keracunan makanan pr...
1198,solo kompas com pemerintah kota pemkot solo ...,solo kompas com pemerintah kota pemkot solo ja...


In [34]:
main_df_berita['stemming_indo'] = main_df_berita['clean_typo_indo'].apply(sastrawistemmer)
stemming=pd.DataFrame(main_df_berita[['clean_typo_indo','stemming_indo']])
stemming

100%|██████████| 235/235 [00:05<00:00, 43.69it/s]


,clean_typo_indo,stemming_indo
0,kompas com menteri pemuda olahraga erick thohi...,kompas com menteri pemuda olahraga erick thohi...
1,kompas com manajer chelsea enzo maresca mengan...,kompas com manajer chelsea enzo maresca anggap...
2,kompas com pelatih liverpool arne slot mengaku...,kompas com latih liverpool arne slot aku cryst...
3,kompas com hasil terbaru pekan liga italia per...,kompas com hasil baru pekan liga italia beda n...
4,kompas com menteri pemuda olahraga republik in...,kompas com menteri pemuda olahraga republik in...
...,...,...
1195,karanganyar kompas com dapur satuan pelayanan ...,karanganyar kompas com dapur satu layan penuh ...
1196,bandung kompas com kepala stasiun geofisika bm...,bandung kompas com kepala stasiun geofisika bm...
1197,tegal kompas com maraknya keracunan makanan pr...,tegal kompas com marak racun makan program mak...
1198,solo kompas com pemerintah kota pemkot solo ja...,solo kompas com perintah kota pemkot solo jawa...


In [35]:
main_df_berita['tokenize_indo'] = main_df_berita['stemming_indo'].apply(tokenize_text)
tokenize=pd.DataFrame(main_df_berita[['stemming_indo','tokenize_indo']])
tokenize

,stemming_indo,tokenize_indo
0,kompas com menteri pemuda olahraga erick thohi...,"[kompas, com, menteri, pemuda, olahraga, erick..."
1,kompas com manajer chelsea enzo maresca anggap...,"[kompas, com, manajer, chelsea, enzo, maresca,..."
2,kompas com latih liverpool arne slot aku cryst...,"[kompas, com, latih, liverpool, arne, slot, ak..."
3,kompas com hasil baru pekan liga italia beda n...,"[kompas, com, hasil, baru, pekan, liga, italia..."
4,kompas com menteri pemuda olahraga republik in...,"[kompas, com, menteri, pemuda, olahraga, repub..."
...,...,...
1195,karanganyar kompas com dapur satu layan penuh ...,"[karanganyar, kompas, com, dapur, satu, layan,..."
1196,bandung kompas com kepala stasiun geofisika bm...,"[bandung, kompas, com, kepala, stasiun, geofis..."
1197,tegal kompas com marak racun makan program mak...,"[tegal, kompas, com, marak, racun, makan, prog..."
1198,solo kompas com perintah kota pemkot solo jawa...,"[solo, kompas, com, perintah, kota, pemkot, so..."


In [36]:
# Simpan DataFrame ke dalam file CSV
main_df_berita.to_csv('datasets/df_berita_preprocessing.csv', index=False)

In [37]:
main_df_berita

,Isi Berita,lwr_indo,clean_sw_indo,clean_stb_indo,clean_typo_indo,stemming_indo,tokenize_indo
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,kompas.com - menteri pemuda dan olahraga erick...,kompas.com - menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick thohi...,kompas com menteri pemuda olahraga erick thohi...,"[kompas, com, menteri, pemuda, olahraga, erick..."
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men...","kompas.com - manajer chelsea, enzo maresca men...","kompas.com - manajer chelsea, enzo maresca men...",kompas com manajer chelsea enzo maresca meng...,kompas com manajer chelsea enzo maresca mengan...,kompas com manajer chelsea enzo maresca anggap...,"[kompas, com, manajer, chelsea, enzo, maresca,..."
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...","kompas.com - pelatih liverpool, arne slot, men...","kompas.com - pelatih liverpool, arne slot, men...",kompas com pelatih liverpool arne slot menga...,kompas com pelatih liverpool arne slot mengaku...,kompas com latih liverpool arne slot aku cryst...,"[kompas, com, latih, liverpool, arne, slot, ak..."
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,kompas.com - hasil terbaru pekan kelima liga i...,kompas.com - hasil terbaru pekan liga italia 2...,kompas com hasil terbaru pekan liga italia ...,kompas com hasil terbaru pekan liga italia per...,kompas com hasil baru pekan liga italia beda n...,"[kompas, com, hasil, baru, pekan, liga, italia..."
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,kompas.com - menteri pemuda dan olahraga repub...,kompas.com - menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik in...,kompas com menteri pemuda olahraga republik in...,"[kompas, com, menteri, pemuda, olahraga, repub..."
...,...,...,...,...,...,...,...
1195,"KARANGANYAR, KOMPAS.com - Dapur Satuan Pelayan...","karanganyar, kompas.com - dapur satuan pelayan...","karanganyar, kompas.com - dapur satuan pelayan...",karanganyar kompas com dapur satuan pelayana...,karanganyar kompas com dapur satuan pelayanan ...,karanganyar kompas com dapur satu layan penuh ...,"[karanganyar, kompas, com, dapur, satu, layan,..."
1196,"BANDUNG, KOMPAS.com -Kepala Stasiun Geofisika ...","bandung, kompas.com -kepala stasiun geofisika ...","bandung, kompas.com -kepala stasiun geofisika ...",bandung kompas com kepala stasiun geofisika b...,bandung kompas com kepala stasiun geofisika bm...,bandung kompas com kepala stasiun geofisika bm...,"[bandung, kompas, com, kepala, stasiun, geofis..."
1197,"TEGAL, KOMPAS.com – Maraknya kasus keracunan m...","tegal, kompas.com – maraknya kasus keracunan m...","tegal, kompas.com – maraknya keracunan makanan...",tegal kompas com maraknya keracunan makanan ...,tegal kompas com maraknya keracunan makanan pr...,tegal kompas com marak racun makan program mak...,"[tegal, kompas, com, marak, racun, makan, prog..."
1198,"SOLO, KOMPAS.com - Pemerintah Kota (Pemkot) So...","solo, kompas.com - pemerintah kota (pemkot) so...","solo, kompas.com - pemerintah kota (pemkot) so...",solo kompas com pemerintah kota pemkot solo ...,solo kompas com pemerintah kota pemkot solo ja...,solo kompas com perintah kota pemkot solo jawa...,"[solo, kompas, com, perintah, kota, pemkot, so..."


In [38]:
df_final_sample.to_csv('datasets/df_berita_200.csv', index=False)